In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier 
from sklearn.metrics import roc_auc_score, classification_report, roc_curve

In [ ]:
data = pd.read_csv("data/RHCsubset.csv")

In [ ]:
data = data.drop(columns=['time'])
data = pd.get_dummies(data, columns=['race', 'income'], drop_first=True)
data = data.astype({col: int for col in data.select_dtypes(include='bool').columns})

In [ ]:
X = data.drop(columns=['death'])
y = data['death']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
models = [
    {'label': 'Random Forest', 'model': RandomForestClassifier(random_state=42)},
    {'label': 'AdaBoost', 'model': AdaBoostClassifier(random_state=42)},
    {'label': 'Gradient Boosting', 'model': GradientBoostingClassifier(random_state=42)},
    {'label': 'Decision Tree', 'model': DecisionTreeClassifier(random_state=42)},
    {'label': 'Logistic Regression', 'model': LogisticRegression(max_iter=10000, random_state=42)},
    {'label': 'MLP Classifier', 'model': MLPClassifier(max_iter=1000, random_state=42)}
]

metrics = {
    'Model': [],
    'Precision': [],
    'Recall': [],
    'F1-Score': []
}

for m in models:
    label = m['label']
    model = m['model']
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    positive_class_metrics = report['1']
    
    metrics['Model'].append(label)
    metrics['Precision'].append(positive_class_metrics['precision'])
    metrics['Recall'].append(positive_class_metrics['recall'])
    metrics['F1-Score'].append(positive_class_metrics['f1-score'])

metrics_df = pd.DataFrame(metrics)

def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: lightgreen' if v else '' for v in is_max]

styled_df = metrics_df.style.apply(highlight_max, subset=['Precision', 'Recall', 'F1-Score'], axis=0)
styled_df.format(precision=2)
styled_df


In [ ]:
models = [
    {'label': 'Random Forest', 'model': RandomForestClassifier(random_state=42)},
    {'label': 'AdaBoost', 'model': AdaBoostClassifier(random_state=42)},
    {'label': 'Gradient Boosting', 'model': GradientBoostingClassifier(random_state=42)},
    {'label': 'Decision Tree', 'model': DecisionTreeClassifier(random_state=42)},
    {'label': 'Logistic Regression', 'model': LogisticRegression(max_iter=10000, random_state=42)},
    {'label': 'MLP Classifier', 'model': MLPClassifier(max_iter=1000, random_state=42)}
]

plt.figure(figsize=(13, 11))

for m in models:
    model = m['model']
    model.fit(X_train, y_train)  # Ensure models are trained
    y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probability predictions
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)  # Calculate FPR and TPR
    auc = roc_auc_score(y_test, y_pred_proba)  # Calculate AUC
    plt.plot(fpr, tpr, label=f"{m['label']} (AUC = {auc:.3f})")

plt.plot([0, 1], [0, 1], '--', c='blue', label='Random Guess (AUC = 0.500)')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.grid(alpha=0)
plt.xticks(size=25)
plt.yticks(size=25)
plt.xlabel('False Positive Rate', fontsize=25, labelpad=10)
plt.ylabel('True Positive Rate', fontsize=25, labelpad=10)
#plt.title('', fontsize=25, pad=20)
plt.legend(loc="lower right", fontsize=15)
plt.savefig("figures/roc_curve.pdf", format='pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import shap

shap_rf = RandomForestClassifier(random_state=42)
shap_rf.fit(X_train, y_train)
explainer_rf = shap.Explainer(shap_rf.predict, X_test)
shap_values_rf = explainer_rf(X_test, max_evals = 500)

shap_gb = GradientBoostingClassifier(random_state=42)
shap_gb.fit(X_train, y_train)
explainer_gb = shap.Explainer(shap_gb.predict, X_test)
shap_values_gb = explainer_gb(X_test, max_evals = 500)

In [ ]:
label = ["RHC", "Age", "Female", "Years of Education", "Temperature", "Weight (Kg)",
        "Glasgow Coma Score", "DAS Index", "Cancer", "Cardiovascular Conditions",
        "Neurological Comorbidities", "Malignancies", "Immunosuppression", "Liver Disease",
        "Psychiatric Conditions", "Congestive Heart Failure", "Renal Conditions", 
        "Other Race", "White", "Income $25k-50k", "Income > $50k", "Income Under $11k"]

In [ ]:
def plot_shap_summary(shap_values, X_test, feature_names, model_name, file_name, 
                      variable_to_highlight="RHC", color_bar=True):
    shap.summary_plot(
        shap_values, X_test, feature_names=feature_names, 
        max_display=15, show=False, plot_size=(13, 12), color_bar=color_bar
    )

    #plt.title(f'{model_name}', fontsize=30, pad=20)
    plt.xlabel('SHAP Value (Impact on model output)', fontsize=25, labelpad=15)
    plt.ylabel('', fontsize=25, labelpad=15)
    plt.xticks(fontsize=25)
    plt.yticks(fontsize=25)

    if color_bar:
        colorbar = plt.gcf().axes[-1]
        colorbar.tick_params(labelsize=25)
        colorbar.set_ylabel('', fontsize=25)

    ax = plt.gca()
    yticks = [tick.get_text() for tick in ax.get_yticklabels()]

    if variable_to_highlight in yticks:
        variable_index = yticks.index(variable_to_highlight)

        rect = patches.Rectangle(
            (-0.6, variable_index - 0.5), 1.2, 1,
            linewidth=2, edgecolor="red", facecolor="yellow", alpha=0.2
        )
        ax.add_patch(rect)
    else:
        print(f"Variable '{variable_to_highlight}' not found in the y-tick labels.")

    plt.savefig(file_name, format='pdf', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
plot_shap_summary(
    shap_values_rf, X_test, label, 
    model_name="Random Forest Classifier", 
    file_name="figures/shap1.pdf", 
    color_bar=False
)

plot_shap_summary(
    shap_values_gb, X_test, label, 
    model_name="Gradient Boosting Model", 
    file_name="figures/shap2.pdf", 
    color_bar=True
)